In [ ]:
# inporting the required liberaries
import numpy as np
# print(numpy.__version__)
import pandas as pd
# print(pd.__version__)
import ast

In [ ]:
# reading the csv files and storing their data in variables
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [5]:
# movies.head(1)

In [6]:
# credits.head(1)

In [ ]:
# merging data of both the variables
movies = movies.merge(credits, on='title')
# movies.head(1)

In [ ]:
#  extracting the required data
# required columns = genres, id, keywords, title, overwiew, cast, crew, 
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
# movies.info()
# movies.head()

In [9]:
# movies.isnull().sum()

In [ ]:
# removing the data entries with NULL values
movies.dropna(inplace=True)

In [11]:
# movies.duplicated().sum()

In [ ]:
# converting in to a list
def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [13]:
movies['genres'] = movies['genres'].apply(convert)

In [14]:
# movies['genres'].head()

In [15]:
# movies.head()

In [16]:
movies['keywords'] = movies['keywords'].apply(convert)
# movies.head()

In [17]:
def convert2(obj):
    l = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter == 3:
            break
        l.append(i['name'])
        counter += 1
    return l
    
movies['cast'] = movies['cast'].apply(convert2)

In [18]:
def findDirector(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l

In [19]:
movies['crew'] = movies['crew'].apply(findDirector)
# movies.head()

In [ ]:
# Rmoving the white space between the words
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [22]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ", "") for i in x])

In [23]:
# movies.head()

In [ ]:
# merging all the columns in a single column called tag
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [25]:
# movies.head()

In [ ]:
# making the new dataframe with the required columns only
new_df = movies[['movie_id', 'title', 'tags']]

In [ ]:
# converting into a stirng
new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))

C:\Users\Abhraneel Singh\AppData\Local\Temp\ipykernel_20440\3530946481.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))


In [ ]:
# coverting into lower case
new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())

C:\Users\Abhraneel Singh\AppData\Local\Temp\ipykernel_20440\3679624391.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())


In [29]:
# new_df['tags'][0]

In [ ]:
# importing nltk library for stemming which is the process of converting words in their root words.
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [31]:
def stem(text):
    y = []

    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)
    

In [32]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\Abhraneel Singh\AppData\Local\Temp\ipykernel_20440\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [ ]:
# importing sklearn library for removing the stopwords and for the process of vectorization by Bag Of Words technique.
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [ ]:

vectors = cv.fit_transform(new_df['tags']).toarray()

In [35]:
# cv.get_feature_names_out()

In [ ]:
# for calculating the angular diatance of all the vectors(movies) from each other
from sklearn.metrics.pairwise import cosine_similarity

In [37]:
similarity = cosine_similarity(vectors)

In [ ]:
# function calculates the nearest 5 movies for a given movies using vectors.
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)),reverse = True, key = lambda x:x[1])[1:6]

    for i in movie_list:
        print(new_df.iloc[i[0]].title)

In [39]:
recommend('Batman')

Batman
Batman & Robin
Batman Begins
Batman Returns
The R.M.


In [ ]:
# importing library for exporting the dataframe and the vector distance. 
import pickle

In [41]:
pickle.dump(new_df, open('movies.pkl', 'wb'))

In [42]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))